#04_DSMarket_SS_calculation

## 1. Import

In [53]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.stats import norm

In [54]:
df_sales = pd.read_parquet("df_forecasting.parquet", engine="pyarrow")
print(df_sales.shape)
df_sales.head(3)

(8354260, 15)


,id,Unnamed: 0,item,category,department,store,store_code,region,yearweek,n_sales,revenue,avg_sell_price,event,cluster_store,cluster_item
0,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201104,0,0.0,NaN,Without event,1,0
1,ACCESORIES_1_001_BOS_1,1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201105,0,0.0,NaN,SuperBowl,1,0
2,ACCESORIES_1_001_BOS_1,2,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201106,0,0.0,NaN,Without event,1,0


In [55]:
df_rmse = pd.read_parquet("rmse_por_id.parquet", engine="pyarrow")
print(df_rmse.shape)
df_rmse.head(3)

(30490, 2)


,id,rmse_mean
0,ACCESORIES_1_001_BOS_1,1.194208
1,ACCESORIES_1_001_BOS_2,0.959239
2,ACCESORIES_1_001_BOS_3,1.713955


In [56]:
df_rmse.head(3)

,id,rmse_mean
0,ACCESORIES_1_001_BOS_1,1.194208
1,ACCESORIES_1_001_BOS_2,0.959239
2,ACCESORIES_1_001_BOS_3,1.713955


In [57]:
df_sales.head(3)

,id,Unnamed: 0,item,category,department,store,store_code,region,yearweek,n_sales,revenue,avg_sell_price,event,cluster_store,cluster_item
0,ACCESORIES_1_001_BOS_1,0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201104,0,0.0,NaN,Without event,1,0
1,ACCESORIES_1_001_BOS_1,1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201105,0,0.0,NaN,SuperBowl,1,0
2,ACCESORIES_1_001_BOS_1,2,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201106,0,0.0,NaN,Without event,1,0


In [ ]:
df_prediction = pd.read_csv("df_prediction_4.csv")  
print(df_prediction.shape)
df_prediction.head(3)


(8476220, 16)


,Unnamed: 0,id,yearweek,avg_sell_price,event,cluster_store,cluster_item,n_sales,date,revenue,item,category,department,store,store_code,region
0,0,ACCESORIES_1_001_BOS_1,201104,NaN,0,1,0,0.0,2011-01-24,0.0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston
1,1,ACCESORIES_1_001_BOS_1,201105,NaN,1,1,0,0.0,2011-01-31,0.0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston
2,2,ACCESORIES_1_001_BOS_1,201106,NaN,0,1,0,0.0,2011-02-07,0.0,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston


In [59]:
df_prediction_sorted = pd.Series(df_prediction["yearweek"].unique()).sort_values()

In [60]:
last_4 = df_prediction_sorted.tail(4)

In [61]:
last_4

274    201617
275    201618
276    201619
277    201620
dtype: int64

In [62]:
df_prediction = df_prediction[df_prediction["yearweek"].isin(last_4)]

In [63]:
df_prediction

,Unnamed: 0,id,yearweek,avg_sell_price,event,cluster_store,cluster_item,n_sales,date,revenue,item,category,department,store,store_code,region
8354260,8354260,ACCESORIES_1_001_BOS_1,201617,10.9858,0,1,0,2.0,2016-04-25,21.9716,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston
8354261,8354261,ACCESORIES_1_001_BOS_1,201618,10.9858,0,1,0,2.0,2016-05-02,21.9716,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston
8354262,8354262,ACCESORIES_1_001_BOS_1,201619,10.9858,0,1,0,2.0,2016-05-09,21.9716,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston
8354263,8354263,ACCESORIES_1_001_BOS_1,201620,10.9858,0,1,0,2.0,2016-05-16,21.9716,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston
8354264,8354264,ACCESORIES_1_001_BOS_2,201617,10.9858,0,3,0,3.0,2016-04-25,32.9574,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Roxbury,BOS_2,Boston
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8476215,8476215,SUPERMARKET_3_827_PHI_2,201620,1.2000,0,0,0,18.0,2016-05-16,21.6000,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Yorktown,PHI_2,Philadelphia
8476216,8476216,SUPERMARKET_3_827_PHI_3,201617,1.2000,0,3,0,6.0,2016-04-25,7.2000,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia
8476217,8476217,SUPERMARKET_3_827_PHI_3,201618,1.2000,0,3,0,9.0,2016-05-02,10.8000,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia
8476218,8476218,SUPERMARKET_3_827_PHI_3,201619,1.2000,0,3,0,10.0,2016-05-09,12.0000,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia


In [64]:
df_prediction_mean = df_prediction.groupby("id")["n_sales"].mean().reset_index()
df_prediction_mean = df_prediction_mean.rename(columns={"n_sales": "DL"})

In [65]:
df_prediction_mean

,id,DL
0,ACCESORIES_1_001_BOS_1,2.00
1,ACCESORIES_1_001_BOS_2,3.00
2,ACCESORIES_1_001_BOS_3,2.75
3,ACCESORIES_1_001_NYC_1,6.25
4,ACCESORIES_1_001_NYC_2,5.00
...,...,...
30485,SUPERMARKET_3_827_NYC_3,33.00
30486,SUPERMARKET_3_827_NYC_4,14.25
30487,SUPERMARKET_3_827_PHI_1,21.75
30488,SUPERMARKET_3_827_PHI_2,19.00


In [66]:
df_rmse = df_rmse.merge(
    df_prediction_mean[['id', 'DL']],  
    on='id',                           
    how='left'                          
)

In [67]:
df_rmse

,id,rmse_mean,DL
0,ACCESORIES_1_001_BOS_1,1.194208,2.00
1,ACCESORIES_1_001_BOS_2,0.959239,3.00
2,ACCESORIES_1_001_BOS_3,1.713955,2.75
3,ACCESORIES_1_001_NYC_1,1.704683,6.25
4,ACCESORIES_1_001_NYC_2,2.850243,5.00
...,...,...,...
30485,SUPERMARKET_3_827_NYC_3,5.868190,33.00
30486,SUPERMARKET_3_827_NYC_4,5.608815,14.25
30487,SUPERMARKET_3_827_PHI_1,8.024147,21.75
30488,SUPERMARKET_3_827_PHI_2,7.191416,19.00


## 2. Cálculo del stock de seguridad

Para calcular el **stock de seguridad**, utilizaremos el **RMSE** obtenido para cada item y tienda durante la fase de predicción.
A partir de este valor, calcularemos el stock de seguridad del producto en cada tienda aplicando la siguiente fórmula:

**SS** = Z ×RMSE ×√L
​
- **Z** es el nivel de servicio deseado, que representa la probabilidad de no tener una rotura de stock durante el tiempo de reposición.
Cuanto mayor sea el nivel de servicio (por ejemplo, 95% o 99%), mayor será el valor de Z y, por tanto, el nivel de protección frente a la variabilidad de la demanda.

- **L** es el lead time o tiempo de aprovisionamiento, es decir, el número de periodos (en este caso semanas) que transcurren desde que se realiza el pedido hasta que la mercancía llega al almacén o tienda.

De esta forma, el stock de seguridad actúa como una reserva adicional que permite absorber la incertidumbre de la demanda y garantizar la disponibilidad del producto incluso en situaciones imprevistas.

In [68]:
service_level = 0.95
z = norm.ppf(service_level)
L = 1 #lead time of one week

In [69]:
z

np.float64(1.6448536269514722)

In [70]:
df_rmse["SS"] = z*df_rmse["rmse_mean"] * np.sqrt(L)

In [71]:
df_rmse

,id,rmse_mean,DL,SS
0,ACCESORIES_1_001_BOS_1,1.194208,2.00,1.964297
1,ACCESORIES_1_001_BOS_2,0.959239,3.00,1.577807
2,ACCESORIES_1_001_BOS_3,1.713955,2.75,2.819205
3,ACCESORIES_1_001_NYC_1,1.704683,6.25,2.803955
4,ACCESORIES_1_001_NYC_2,2.850243,5.00,4.688232
...,...,...,...,...
30485,SUPERMARKET_3_827_NYC_3,5.868190,33.00,9.652314
30486,SUPERMARKET_3_827_NYC_4,5.608815,14.25,9.225679
30487,SUPERMARKET_3_827_PHI_1,8.024147,21.75,13.198548
30488,SUPERMARKET_3_827_PHI_2,7.191416,19.00,11.828827


## 2. ROP — nivel de pedido (Reorder Point)

El punto de pedido (ROP) indica el nivel de inventario en el que se debe realizar un nuevo pedido para evitar una rotura de stock, y se calcula de la siguiente manera:

𝑅𝑂𝑃 =𝐷𝐿 +𝑆𝑆

donde:

**DL** = demanda media durante el lead time, que en nuestro caso utilizaremos como la demanda media del artículo en la tienda

**SS** = stock de seguridad

In [72]:
df_sales["year"] = df_sales["yearweek"].astype(str).str[:4].astype(int)

In [73]:
df_sales["n_sales_sum"] = df_sales.groupby("id")["n_sales"].transform("sum")

In [74]:
df_sales["n_sales_sum_2015"] = df_sales[df_sales["year"] == 2015].groupby("id")["n_sales"].transform("sum")

In [75]:
df_sales = df_sales.dropna(subset=["n_sales_sum_2015"]) #Eliminar las filas nulas que se generan

In [76]:
df_sales["revenue"] = df_sales["n_sales"] * df_sales["avg_sell_price"]

In [77]:
df_sales["revenue_sum"] = df_sales.groupby("id")["revenue"].transform("sum")

In [78]:
df_sales["avg_price"] = df_sales["revenue_sum"] / df_sales["n_sales_sum"]

In [79]:
columnas = ["id","item","store_code","store","category","avg_price","n_sales_sum_2015"]

In [80]:
df_sales = df_sales[columnas]

In [81]:
df_sales.shape

(1615970, 7)

In [82]:
df_sales = df_sales.drop_duplicates()

In [83]:
df_sales.shape

(30490, 7)

In [84]:
df_sales.columns

Index(['id', 'item', 'store_code', 'store', 'category', 'avg_price',
       'n_sales_sum_2015'],
      dtype='object')

In [ ]:
df_final = df_sales.merge(
    df_rmse[['id','rmse_mean','SS','DL']],  
    on=['id'],
    how='right'  
)

In [86]:
df_final

,id,item,store_code,store,category,avg_price,n_sales_sum_2015,rmse_mean,SS,DL
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,South_End,ACCESORIES,3.642665,87.0,1.194208,1.964297,2.00
1,ACCESORIES_1_001_BOS_2,ACCESORIES_1_001,BOS_2,Roxbury,ACCESORIES,5.000999,183.0,0.959239,1.577807,3.00
2,ACCESORIES_1_001_BOS_3,ACCESORIES_1_001,BOS_3,Back_Bay,ACCESORIES,5.421564,190.0,1.713955,2.819205,2.75
3,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,NYC_1,Greenwich_Village,ACCESORIES,4.119675,225.0,1.704683,2.803955,6.25
4,ACCESORIES_1_001_NYC_2,ACCESORIES_1_001,NYC_2,Harlem,ACCESORIES,3.648421,180.0,2.850243,4.688232,5.00
...,...,...,...,...,...,...,...,...,...,...
30485,SUPERMARKET_3_827_NYC_3,SUPERMARKET_3_827,NYC_3,Tribeca,SUPERMARKET,0.397978,853.0,5.868190,9.652314,33.00
30486,SUPERMARKET_3_827_NYC_4,SUPERMARKET_3_827,NYC_4,Brooklyn,SUPERMARKET,0.012903,1.0,5.608815,9.225679,14.25
30487,SUPERMARKET_3_827_PHI_1,SUPERMARKET_3_827,PHI_1,Midtown_Village,SUPERMARKET,0.575842,1311.0,8.024147,13.198548,21.75
30488,SUPERMARKET_3_827_PHI_2,SUPERMARKET_3_827,PHI_2,Yorktown,SUPERMARKET,0.636678,460.0,7.191416,11.828827,19.00


In [87]:
df_final["ROP"] = (df_final["DL"] + df_final["SS"]).round(0)

In [88]:
df_final["SS"] = df_final["SS"].round(0)

In [89]:
df_final

,id,item,store_code,store,category,avg_price,n_sales_sum_2015,rmse_mean,SS,DL,ROP
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,South_End,ACCESORIES,3.642665,87.0,1.194208,2.0,2.00,4.0
1,ACCESORIES_1_001_BOS_2,ACCESORIES_1_001,BOS_2,Roxbury,ACCESORIES,5.000999,183.0,0.959239,2.0,3.00,5.0
2,ACCESORIES_1_001_BOS_3,ACCESORIES_1_001,BOS_3,Back_Bay,ACCESORIES,5.421564,190.0,1.713955,3.0,2.75,6.0
3,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,NYC_1,Greenwich_Village,ACCESORIES,4.119675,225.0,1.704683,3.0,6.25,9.0
4,ACCESORIES_1_001_NYC_2,ACCESORIES_1_001,NYC_2,Harlem,ACCESORIES,3.648421,180.0,2.850243,5.0,5.00,10.0
...,...,...,...,...,...,...,...,...,...,...,...
30485,SUPERMARKET_3_827_NYC_3,SUPERMARKET_3_827,NYC_3,Tribeca,SUPERMARKET,0.397978,853.0,5.868190,10.0,33.00,43.0
30486,SUPERMARKET_3_827_NYC_4,SUPERMARKET_3_827,NYC_4,Brooklyn,SUPERMARKET,0.012903,1.0,5.608815,9.0,14.25,23.0
30487,SUPERMARKET_3_827_PHI_1,SUPERMARKET_3_827,PHI_1,Midtown_Village,SUPERMARKET,0.575842,1311.0,8.024147,13.0,21.75,35.0
30488,SUPERMARKET_3_827_PHI_2,SUPERMARKET_3_827,PHI_2,Yorktown,SUPERMARKET,0.636678,460.0,7.191416,12.0,19.00,31.0


## 3. EOQ (Economic Order Quantity)

El **EOQ** (Economic Order Quantity) o Cantidad Económica de Pedido es un modelo de gestión de inventarios que ayuda a las empresas a decidir cuántas unidades deben pedir cada vez para mantener los costos lo más bajos posible.

El objetivo es encontrar un equilibrio entre:

- Pedir demasiado a menudo, lo que aumenta los costos de pedido, y

- Pedir grandes cantidades, lo que incrementa los costos de almacenamiento.

En resumen, el EOQ permite **optimizar la gestión de inventarios**, garantizando que haya suficiente stock para satisfacer la demanda sin generar costos innecesarios

La fórmula clásica del EOQ es:

𝐸𝑂𝑄 = √(2𝐷⋅S/𝐻)

donde:

**D** = demanda anual (unidades por año)

**S** = costo por pedido (costo de preparación o de orden)

**H** = costo de mantenimiento por unidad al año (holding cost per unit per year)

Esta fórmula determina la cantidad óptima de pedido que minimiza el costo total de inventario, es decir, la suma de los costos de pedido y los costos de mantenimiento.

Dado que ni **H** ni **S** están disponibles para este problema, vamos a **suponer sus valores en función de la categoría de los artículos**.
En particular, **H**, el costo de mantenimiento en almacén, se calculará como un **porcentaje del precio medio de venta** del producto, de la siguiente manera:

* **SUPERMARKET:** 15 % → productos que se venden más rápido, con mayor rotación y menos tiempo en almacén.
* **HOME & GARDEN:** 20 % → productos con una rotación menor.
* **ACCESORIES:** 25 % → productos que se venden con menos frecuencia, por lo tanto, con un mayor porcentaje de costo en caso de pedido.


In [90]:
h_dict = {
    'SUPERMARKET': 0.15,
    'HOME_&_GARDEN': 0.20,
    'ACCESORIES': 0.25
}

In [91]:
df_final['H_%'] = df_final['category'].map(h_dict)

In [92]:
df_final[df_final['category']=="ACCESORIES"].head()

,id,item,store_code,store,category,avg_price,n_sales_sum_2015,rmse_mean,SS,DL,ROP,H_%
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,South_End,ACCESORIES,3.642665,87.0,1.194208,2.0,2.00,4.0,0.25
1,ACCESORIES_1_001_BOS_2,ACCESORIES_1_001,BOS_2,Roxbury,ACCESORIES,5.000999,183.0,0.959239,2.0,3.00,5.0,0.25
2,ACCESORIES_1_001_BOS_3,ACCESORIES_1_001,BOS_3,Back_Bay,ACCESORIES,5.421564,190.0,1.713955,3.0,2.75,6.0,0.25
3,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,NYC_1,Greenwich_Village,ACCESORIES,4.119675,225.0,1.704683,3.0,6.25,9.0,0.25
4,ACCESORIES_1_001_NYC_2,ACCESORIES_1_001,NYC_2,Harlem,ACCESORIES,3.648421,180.0,2.850243,5.0,5.00,10.0,0.25


In [93]:
df_final['H']=df_final['H_%']*df_final['avg_price']

In [94]:
mean_H = df_final["H"].mean()
mean_H

np.float64(0.30803362477529217)

In [95]:
df_final.loc[df_final['H'] == 0, 'H'] = mean_H

In [96]:
df_final

,id,item,store_code,store,category,avg_price,n_sales_sum_2015,rmse_mean,SS,DL,ROP,H_%,H
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,South_End,ACCESORIES,3.642665,87.0,1.194208,2.0,2.00,4.0,0.25,0.910666
1,ACCESORIES_1_001_BOS_2,ACCESORIES_1_001,BOS_2,Roxbury,ACCESORIES,5.000999,183.0,0.959239,2.0,3.00,5.0,0.25,1.250250
2,ACCESORIES_1_001_BOS_3,ACCESORIES_1_001,BOS_3,Back_Bay,ACCESORIES,5.421564,190.0,1.713955,3.0,2.75,6.0,0.25,1.355391
3,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,NYC_1,Greenwich_Village,ACCESORIES,4.119675,225.0,1.704683,3.0,6.25,9.0,0.25,1.029919
4,ACCESORIES_1_001_NYC_2,ACCESORIES_1_001,NYC_2,Harlem,ACCESORIES,3.648421,180.0,2.850243,5.0,5.00,10.0,0.25,0.912105
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,SUPERMARKET_3_827_NYC_3,SUPERMARKET_3_827,NYC_3,Tribeca,SUPERMARKET,0.397978,853.0,5.868190,10.0,33.00,43.0,0.15,0.059697
30486,SUPERMARKET_3_827_NYC_4,SUPERMARKET_3_827,NYC_4,Brooklyn,SUPERMARKET,0.012903,1.0,5.608815,9.0,14.25,23.0,0.15,0.001935
30487,SUPERMARKET_3_827_PHI_1,SUPERMARKET_3_827,PHI_1,Midtown_Village,SUPERMARKET,0.575842,1311.0,8.024147,13.0,21.75,35.0,0.15,0.086376
30488,SUPERMARKET_3_827_PHI_2,SUPERMARKET_3_827,PHI_2,Yorktown,SUPERMARKET,0.636678,460.0,7.191416,12.0,19.00,31.0,0.15,0.095502


https://www.ascendsoftware.com/blog/the-average-cost-of-processing-a-purchase-order-a-detailed-analysis?utm_source=chatgpt.com

https://it.scribd.com/document/459659976/2019-HCKT-Metrics-Procurement-Cost-v1?utm_source=chatgpt.com

Dado que el costo por pedido (S) no está disponible, supondremos tres escenarios posibles aplicables a todas las categorías de productos:

Escenario mínimo: 10 € por pedido → procesos muy automatizados o simples.

Escenario medio: 20 € por pedido → nivel medio de digitalización y complejidad.

Escenario alto: 40 € por pedido → procesos manuales o con mayor carga administrativa y de control.

In [97]:
df_final["S"] = 20

In [98]:
df_final

,id,item,store_code,store,category,avg_price,n_sales_sum_2015,rmse_mean,SS,DL,ROP,H_%,H,S
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,BOS_1,South_End,ACCESORIES,3.642665,87.0,1.194208,2.0,2.00,4.0,0.25,0.910666,20
1,ACCESORIES_1_001_BOS_2,ACCESORIES_1_001,BOS_2,Roxbury,ACCESORIES,5.000999,183.0,0.959239,2.0,3.00,5.0,0.25,1.250250,20
2,ACCESORIES_1_001_BOS_3,ACCESORIES_1_001,BOS_3,Back_Bay,ACCESORIES,5.421564,190.0,1.713955,3.0,2.75,6.0,0.25,1.355391,20
3,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,NYC_1,Greenwich_Village,ACCESORIES,4.119675,225.0,1.704683,3.0,6.25,9.0,0.25,1.029919,20
4,ACCESORIES_1_001_NYC_2,ACCESORIES_1_001,NYC_2,Harlem,ACCESORIES,3.648421,180.0,2.850243,5.0,5.00,10.0,0.25,0.912105,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,SUPERMARKET_3_827_NYC_3,SUPERMARKET_3_827,NYC_3,Tribeca,SUPERMARKET,0.397978,853.0,5.868190,10.0,33.00,43.0,0.15,0.059697,20
30486,SUPERMARKET_3_827_NYC_4,SUPERMARKET_3_827,NYC_4,Brooklyn,SUPERMARKET,0.012903,1.0,5.608815,9.0,14.25,23.0,0.15,0.001935,20
30487,SUPERMARKET_3_827_PHI_1,SUPERMARKET_3_827,PHI_1,Midtown_Village,SUPERMARKET,0.575842,1311.0,8.024147,13.0,21.75,35.0,0.15,0.086376,20
30488,SUPERMARKET_3_827_PHI_2,SUPERMARKET_3_827,PHI_2,Yorktown,SUPERMARKET,0.636678,460.0,7.191416,12.0,19.00,31.0,0.15,0.095502,20


In [99]:
df_final['EOQ'] = np.round(np.sqrt((2 * df_final['DL'] * df_final['S']) / df_final['H']))

In [100]:
filtered_df = df_final[
    (df_final['EOQ'].isin([0, np.inf])) 
]

In [101]:
mean_H = df_final["H"].mean()
mean_H

np.float64(0.3095995549766752)

In [102]:
df_final[df_final["DL"]>50]

,id,item,store_code,store,category,avg_price,n_sales_sum_2015,rmse_mean,SS,DL,ROP,H_%,H,S,EOQ
73,ACCESORIES_1_008_NYC_1,ACCESORIES_1_008,NYC_1,Greenwich_Village,ACCESORIES,0.158750,3480.0,23.853753,39.0,55.00,94.0,0.25,0.039687,20,235.0
75,ACCESORIES_1_008_NYC_3,ACCESORIES_1_008,NYC_3,Tribeca,ACCESORIES,0.093514,2467.0,12.616720,21.0,57.50,78.0,0.25,0.023379,20,314.0
183,ACCESORIES_1_019_NYC_1,ACCESORIES_1_019,NYC_1,Greenwich_Village,ACCESORIES,0.182732,2626.0,11.935394,20.0,54.50,74.0,0.25,0.045683,20,218.0
473,ACCESORIES_1_048_NYC_1,ACCESORIES_1_048,NYC_1,Greenwich_Village,ACCESORIES,0.197485,4336.0,28.235357,46.0,93.00,139.0,0.25,0.049371,20,274.0
475,ACCESORIES_1_048_NYC_3,ACCESORIES_1_048,NYC_3,Tribeca,ACCESORIES,0.208542,3541.0,20.235862,33.0,63.25,97.0,0.25,0.052135,20,220.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30324,SUPERMARKET_3_811_NYC_2,SUPERMARKET_3_811,NYC_2,Harlem,SUPERMARKET,0.484056,1818.0,15.356740,25.0,61.50,87.0,0.15,0.072608,20,184.0
30325,SUPERMARKET_3_811_NYC_3,SUPERMARKET_3_811,NYC_3,Tribeca,SUPERMARKET,0.461241,5200.0,30.037044,49.0,88.50,138.0,0.15,0.069186,20,226.0
30328,SUPERMARKET_3_811_PHI_2,SUPERMARKET_3_811,PHI_2,Yorktown,SUPERMARKET,0.528582,6921.0,23.408332,39.0,206.25,245.0,0.15,0.079287,20,323.0
30329,SUPERMARKET_3_811_PHI_3,SUPERMARKET_3_811,PHI_3,Queen_Village,SUPERMARKET,0.528442,3864.0,26.006997,43.0,157.00,200.0,0.15,0.079266,20,281.0


In [103]:
df_final[df_final["item"]=='ACCESORIES_1_008']

,id,item,store_code,store,category,avg_price,n_sales_sum_2015,rmse_mean,SS,DL,ROP,H_%,H,S,EOQ
70,ACCESORIES_1_008_BOS_1,ACCESORIES_1_008,BOS_1,South_End,ACCESORIES,0.103346,931.0,5.694226,9.0,14.00,23.0,0.25,0.025837,20,147.0
71,ACCESORIES_1_008_BOS_2,ACCESORIES_1_008,BOS_2,Roxbury,ACCESORIES,0.179666,2853.0,9.607944,16.0,22.50,38.0,0.25,0.044917,20,142.0
72,ACCESORIES_1_008_BOS_3,ACCESORIES_1_008,BOS_3,Back_Bay,ACCESORIES,0.108499,850.0,4.799702,8.0,17.75,26.0,0.25,0.027125,20,162.0
73,ACCESORIES_1_008_NYC_1,ACCESORIES_1_008,NYC_1,Greenwich_Village,ACCESORIES,0.158750,3480.0,23.853753,39.0,55.00,94.0,0.25,0.039687,20,235.0
74,ACCESORIES_1_008_NYC_2,ACCESORIES_1_008,NYC_2,Harlem,ACCESORIES,0.124100,1554.0,14.563471,24.0,27.50,51.0,0.25,0.031025,20,188.0
75,ACCESORIES_1_008_NYC_3,ACCESORIES_1_008,NYC_3,Tribeca,ACCESORIES,0.093514,2467.0,12.616720,21.0,57.50,78.0,0.25,0.023379,20,314.0
76,ACCESORIES_1_008_NYC_4,ACCESORIES_1_008,NYC_4,Brooklyn,ACCESORIES,0.123837,1791.0,3.771765,6.0,31.50,38.0,0.25,0.030959,20,202.0
77,ACCESORIES_1_008_PHI_1,ACCESORIES_1_008,PHI_1,Midtown_Village,ACCESORIES,0.098829,1425.0,7.309262,12.0,25.50,38.0,0.25,0.024707,20,203.0
78,ACCESORIES_1_008_PHI_2,ACCESORIES_1_008,PHI_2,Yorktown,ACCESORIES,0.085633,773.0,5.012218,8.0,17.75,26.0,0.25,0.021408,20,182.0
79,ACCESORIES_1_008_PHI_3,ACCESORIES_1_008,PHI_3,Queen_Village,ACCESORIES,0.097383,737.0,4.205831,7.0,16.00,23.0,0.25,0.024346,20,162.0


In [ ]:
df_final["item"] == ""

In [104]:
df_final.to_csv("df_final_SS.csv", index=False, encoding="utf-8")